# Script to analyse and visualise memory usage

In [ ]:
import linecache
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')
import tracemalloc as tm

In [ ]:
#run_name='python_gc'
run_name='python'
file_profile='output/'+run_name+'_profile_mem.pkl'
file_png_prefix='anal/figs/'+run_name+'_'

In [ ]:
def display_top(snapshot, key_type='lineno', limit=10):
    snapshot = snapshot.filter_traces((
        tm.Filter(False, "<frozen importlib._bootstrap>"),
        tm.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        print("#%s: %s:%s: %.1f KiB"
              % (index, frame.filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

In [ ]:
with open(file_profile,'rb') as f:
    mem_data=pickle.load(f)
    mem_snap=pickle.load(f)
display_top(mem_snap,limit=20)    
#print('mem_data =',mem_data)

In [ ]:
# Plot (graph) size and peak size over time
# Extract graph exectution times: there must be an easier way than this!
graph_mask=list(key.find('Graph')>=0 for key in mem_data.keys())
graph_keys=np.array(list(key for key in mem_data.keys()))[graph_mask]
graph_mem_size=np.array(list(mem_data[graph_keys[i]]['mem_size'][0] for i in range(len(graph_keys))))
graph_mem_peak=np.array(list(mem_data[graph_keys[i]]['mem_peak'][0] for i in range(len(graph_keys))))

n_graph=len(graph_mem_size)
plt.figure(figsize=[12,8])
plt.semilogy(np.arange(n_graph),graph_mem_size/1e6,'-',label='size')
plt.plot(np.arange(n_graph),graph_mem_peak/1e6,'-',label='peak')
plt.title('Memory usage during graph traversal')
plt.xlabel(r'Graph number)')
plt.ylabel(r'Graph $\log_{10}$(Memory usage / MB)')
plt.legend()
plt.savefig(file_png_prefix+'mem.png',bbox_inches='tight')